In [2]:
import os
import time
import numpy as np
import soundfile as sf
import librosa
import csv
import scipy.misc
import imageio
import shutil
import sys
import datetime
import random
import collections
import pickle
import math
from random import randint

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

from keras.layers import Dropout, LSTM,Reshape, Bidirectional, Concatenate, RepeatVector, Permute, Dot, Multiply
from keras.layers import Dense, Activation, Flatten, MaxPooling2D, Input, BatchNormalization,Conv2D,AveragePooling2D
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import losses, optimizers

from keras.layers.pooling import GlobalMaxPooling1D

In [3]:
with open("datasets/train_5000_gunshot_Prob1_44100_128_framelevel_calssifier.pkl", "rb") as input_file:
    train_data = pickle.load(input_file)
input_file.close()

In [4]:
train_frames = train_data['train_frames']
train_labels = train_data['train_labels']

In [5]:
with open("datasets/val_1000_gunshot_44100_128_framelevel_calssifier.pkl", "rb") as input_file:
    val_data = pickle.load(input_file)
input_file.close()

In [6]:
val_frames = val_data['val_frames']
val_labels = val_data['val_labels']

In [7]:
train_frames = np.array(train_frames).reshape(5000,128,200,1)
val_frames = np.array(val_frames).reshape(500,128,200,1)
train_labels = np.array(train_labels).reshape(5000,200)
val_labels = np.array(val_labels).reshape(500,200)

In [8]:
def get_1d_conv_model():
    
    inp = Input(shape=(128,200,1))
    
    x = Conv2D(64, (10,1), strides=(1,1) , padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((10,1),strides=(10,1))(x)
    x = Dropout(rate=0.1)(x)
    print(x._keras_shape)
    
    x = Conv2D(256, (7,1), strides=(1,1) , padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((7,1),strides=(7,1))(x)
    x = Dropout(rate=0.1)(x)
    print(x._keras_shape)
    
    
    x = Reshape((256,200))(x)
    
    x = LSTM(200, return_sequences=True, return_state=True)(x)
    
    x = LSTM(200, return_sequences=True)(x)
    
    
    print(x._keras_shape)
    x = Flatten()(x)
    out = Dense(200, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(lr=0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model

In [22]:
my_model = get_1d_conv_model()

(None, 12, 200, 64)
(None, 1, 200, 256)
(None, 256, 200)


In [23]:
my_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 200, 1)  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 128, 200, 64) 704         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 128, 200, 64) 256         conv2d_3[0][0]                   
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 128, 200, 64) 0           batch_normalization_3[0][0]      
__________________________________________________________________________________________________
max_poolin

In [24]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=10, mode='auto')

In [25]:
my_model.fit(x=train_frames,y=train_labels,batch_size=150,epochs=100, 
             validation_data=(val_frames, val_labels),callbacks=[earlystop])

Train on 5000 samples, validate on 500 samples
Epoch 1/100
5000/5000 [==============================] - 57s 11ms/step - loss: 0.6191 - acc: 0.6958 - val_loss: 0.4187 - val_acc: 0.8176
Epoch 2/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.3555 - acc: 0.8454 - val_loss: 0.3259 - val_acc: 0.8620
Epoch 3/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.2963 - acc: 0.8738 - val_loss: 0.4260 - val_acc: 0.8103
Epoch 4/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.2726 - acc: 0.8828 - val_loss: 0.3254 - val_acc: 0.8620
Epoch 5/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.2550 - acc: 0.8898 - val_loss: 0.2861 - val_acc: 0.8808
Epoch 6/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.2405 - acc: 0.8951 - val_loss: 0.3131 - val_acc: 0.8756
Epoch 7/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.2354 - acc: 0.8985 - val_loss: 0.3042 - val_acc: 0

5000/5000 [==============================] - 56s 11ms/step - loss: 0.0077 - acc: 0.9977 - val_loss: 0.2056 - val_acc: 0.9487
Epoch 61/100
5000/5000 [==============================] - 56s 11ms/step - loss: 0.0072 - acc: 0.9978 - val_loss: 0.2117 - val_acc: 0.9469


In [12]:
my_model = load_model('final_melspec_gunshot_model_2cnn_64_256_2lstm_1dense_200_01.h5')

In [4]:
with open('test_set_3_500_gunshot_44100_128_multiple_estimated_37_preprocessed_with_confidence0.6.txt','r') as regions_file:
    reader = (regions_file.read())
    l = list(reader.split('\n'))
regions_file.close()

In [5]:
len(l)

809

In [8]:
#my_model.save('final_melspec_gunshot_model_2cnn_64_256_2lstm_1dense_200_05.h5')

In [6]:
frames_per_sample = 200

def create_melspec_test(path,start_time,finish_time):
    test_frames=[]
    
    melspec = imageio.imread(path)
    #print(path)
    event_start_index = int(np.round(float(start_time)/30 * 1305))
    event_finish_index = int(np.round(float(finish_time)/30 * 1305))
    event_length = event_finish_index - event_start_index 
    #print(event_start_index)
    #print(event_finish_index)
                    
    #calculating sample start and finish indices, randomly
    sample_start = randint(max(0,event_start_index - max(0,(200-event_length))),event_start_index)
    sample_finish = min(1305,sample_start+frames_per_sample)
                    
    #print('before')
    #print(sample_start)
    #print(sample_finish)
    sample_length = sample_finish - sample_start 
    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
        sample_start = sample_start - (frames_per_sample - sample_length)
                    
    #print('after')
    #print(sample_start)
    #print(sample_finish)
                    
    temp1 = np.array(melspec)
    temp_test_frames = temp1[:,sample_start:sample_finish]
                    
    #print(np.array(temp_test_frames).shape)
    
    temp2 = np.array(temp_test_frames)
    temp2 = temp2.reshape(128,200)
    test_frames = temp2
                    
    #print(np.array(test_frames).shape)
    
    
    return test_frames

In [7]:
def choose_most_probable_event(dic):
    delimeter = '\t'
    ind = 0
    max_val = 0
    max_above1 = 0
    
    for i in range(len(dic['predicted_values'])):
        print(dic['file_name'][i] + ',' + str(dic['event_start_time'][ind]) + ',' + str(dic['event_end_time'][ind]))
        temp_list = dic['predicted_values'][i][0]
        conf = dic['maskrcnn_conf'][i]
        #print(temp_list)
        print('conf is '+str(conf))
        
        m = max(temp_list)
        sum_above1 = 0
        num_above1 = 0
        for j in temp_list:
            if j>=0.1:
                sum_above1 += j
                num_above1 += 1
        if num_above1 > 0:
            avg_above1 = float(sum_above1/num_above1)
        else:
            avg_above1 =0 
            
        if (0.7*(avg_above1) + 0.3*(conf)) > max_val:
            max_val = 0.7*(avg_above1) + 0.3*(conf)
            max_above1 = avg_above1
            max_m = m
            ind = i
        print('avg above 1 is '+str(avg_above1))
             
        
    if max_val >= 0.8:
        st = dic['file_name'][ind]+ delimeter + str(dic['event_start_time'][ind]) + delimeter + str(dic['event_end_time'][ind]) + delimeter + event
    else:
        st = dic['file_name'][ind]+ delimeter + '0.0\t0.0\tno_event'
    return st
    

In [8]:
logs_path = 'logs/'
log_path = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
log_path = os.path.join(logs_path, log_path)

if os.path.exists(log_path):
    pass
else:
    os.mkdir(log_path)
shutil.copyfile('LogMel_Classifier-with_context_gunshot.ipynb', os.path.join(log_path, 'logmel_classifier_code.ipynb'))
 

'logs/2018-12-19-18-14/logmel_classifier_code.ipynb'

In [13]:
event = 'gunshot'
test_path = '../Maskrcnn_audio/datasets/test_set_3_gunshot/'
previous_file_name = l[0].split(';')[0].replace('[','').replace("'",'')
dic = collections.defaultdict(list)

'''
Change the path before running it next time!!!!
'''
#with open(os.path.join(log_path+'/final_report.txt'),'w') as w_file:
with open(os.path.join(log_path + '/final_report.txt'),'w') as w_file:
    for element in l:
        parts = element.split(';')
        
        if(parts[3]==event):
            current_file_name = parts[0].replace('[','').replace("'",'')
            start_time = float(parts[1])
            finish_time = float(parts[2])
            
            
            test_frames = create_melspec_test(test_path + current_file_name +'.png', start_time, finish_time)
            test_frames = np.array(test_frames)
            test_frames = test_frames.reshape(1,128,200,1)
            #print(test_frames.shape)
            
            predictions = my_model.predict(x=test_frames)
            
            #print(parts[0]+ parts[1]+parts[2])
            #print(predictions)
                    
            predict_values = np.array(predictions)
            if current_file_name == previous_file_name:
                dic['file_name'].append(current_file_name)
                dic['event_start_time'].append(float(parts[1]))
                dic['event_end_time'].append(float(parts[2]))
                dic['predicted_values'].append(predict_values)
                dic['maskrcnn_conf'].append(float(parts[4]))
            else:
                
                st = choose_most_probable_event(dic)
                w_file.write(st+'\n')
                dic.clear()
                previous_file_name = current_file_name
                dic['file_name'].append(current_file_name)
                dic['event_start_time'].append(float(parts[1]))
                dic['event_end_time'].append(float(parts[2]))
                dic['predicted_values'].append(predict_values)
                dic['maskrcnn_conf'].append(float(parts[4]))
        else:
            w_file.write(element+'\n')
    #for the last file
    st = choose_most_probable_event(dic)
    w_file.write(st+'\n')
    dic.clear()    
            
w_file.close()

mixture_devtrain_gunshot_000_7b3241e096ea6ab86c68497093fa6466.wav,0.45977011494252873,2.5057471264367814
conf is 0.9774305
avg above 1 is 0.9835959681780031
mixture_devtrain_gunshot_001_2c32b9997d6e192bfd05db7442897030.wav,14.988505747126435,15.74712643678161
conf is 0.993849
avg above 1 is 0.7232952811049692
mixture_devtrain_gunshot_001_2c32b9997d6e192bfd05db7442897030.wav,14.988505747126435,15.74712643678161
conf is 0.74920523
avg above 1 is 0.3172837613245188
mixture_devtrain_gunshot_001_2c32b9997d6e192bfd05db7442897030.wav,14.988505747126435,15.74712643678161
conf is 0.64104205
avg above 1 is 0.5110735114739866
mixture_devtrain_gunshot_001_2c32b9997d6e192bfd05db7442897030.wav,14.988505747126435,15.74712643678161
conf is 0.83990306
avg above 1 is 0.7343049285023712
mixture_devtrain_gunshot_003_205e2070fe11dac293c3f63e0f7f32b4.wav,11.471264367816092,12.344827586206897
conf is 0.9374955
avg above 1 is 0.5422329968178129
mixture_devtrain_gunshot_004_b65d9c3e37da36a4381f7e96181783e0.wav

mixture_devtrain_gunshot_036_d695eb55c6a52ae2c0abc43be0354945.wav,27.28735632183908,28.988505747126435
conf is 0.99849546
avg above 1 is 0.9895459503408462
mixture_devtrain_gunshot_038_31493c51419c3ac79e3fa4cc858cf582.wav,1.4482758620689655,1.8620689655172413
conf is 0.99374515
avg above 1 is 0.4587888980141053
mixture_devtrain_gunshot_038_31493c51419c3ac79e3fa4cc858cf582.wav,1.4482758620689655,1.8620689655172413
conf is 0.7152931
avg above 1 is 0.7588932613531748
mixture_devtrain_gunshot_038_31493c51419c3ac79e3fa4cc858cf582.wav,16.620689655172416,18.27586206896552
conf is 0.93866175
avg above 1 is 0.8423151419560114
mixture_devtrain_gunshot_039_645242d8c0f267b2fc907a2677f18859.wav,16.091954022988507,17.103448275862068
conf is 0.9919008
avg above 1 is 0.8963447171060935
mixture_devtrain_gunshot_039_645242d8c0f267b2fc907a2677f18859.wav,16.091954022988507,17.103448275862068
conf is 0.8362842
avg above 1 is 0.4500249446584628
mixture_devtrain_gunshot_040_db4ff2ba2586c26c74eac937f9203bdd.w

mixture_devtrain_gunshot_086_8912b62be75b783a78ebf4d36ace2974.wav,12.183908045977011,12.528735632183908
conf is 0.7348007
avg above 1 is 0.7414969170861172
mixture_devtrain_gunshot_086_8912b62be75b783a78ebf4d36ace2974.wav,12.183908045977011,12.528735632183908
conf is 0.5037858
avg above 1 is 0.31307591394417816
mixture_devtrain_gunshot_087_1307676b3b8081764ed3a6fce6121b84.wav,3.908045977011494,4.666666666666667
conf is 0.98981744
avg above 1 is 0.28381063585931604
mixture_devtrain_gunshot_089_149f2ef59f6cb910815e7147984a9381.wav,21.724137931034484,23.24137931034483
conf is 0.99135274
avg above 1 is 0.9734724543311379
mixture_devtrain_gunshot_089_149f2ef59f6cb910815e7147984a9381.wav,21.724137931034484,23.24137931034483
conf is 0.5992172
avg above 1 is 0.2613465346788105
mixture_devtrain_gunshot_089_149f2ef59f6cb910815e7147984a9381.wav,21.724137931034484,23.24137931034483
conf is 0.68504006
avg above 1 is 0.49030433252856537
mixture_devtrain_gunshot_090_0448a44690c3c9ab6ef86feb10e6ec60.w

avg above 1 is 0.2388495929977473
mixture_devtrain_gunshot_139_f3529c5132b5d652a612b255a7861203.wav,18.919540229885058,20.13793103448276
conf is 0.54687816
avg above 1 is 0.503533041811195
mixture_devtrain_gunshot_139_f3529c5132b5d652a612b255a7861203.wav,18.919540229885058,20.13793103448276
conf is 0.5586443
avg above 1 is 0.7354349887942615
mixture_devtrain_gunshot_139_f3529c5132b5d652a612b255a7861203.wav,2.8045977011494254,3.218390804597701
conf is 0.93729675
avg above 1 is 0.5822457107197907
mixture_devtrain_gunshot_139_f3529c5132b5d652a612b255a7861203.wav,2.9195402298850572,3.747126436781609
conf is 0.7502682
avg above 1 is 0.4796810986267196
mixture_devtrain_gunshot_139_f3529c5132b5d652a612b255a7861203.wav,2.9195402298850572,3.747126436781609
conf is 0.8461623
avg above 1 is 0.49904272315048037
mixture_devtrain_gunshot_140_094297e9b2fa2fd214d91ee7221e43f7.wav,16.344827586206897,17.74712643678161
conf is 0.99684143
avg above 1 is 0.9905192220681592
mixture_devtrain_gunshot_141_b076

mixture_devtrain_gunshot_174_b724ab68f6f8a406b7f4977f178b6fca.wav,6.436781609195402,7.011494252873563
conf is 0.9988933
avg above 1 is 0.8833748331587565
mixture_devtrain_gunshot_176_566d617826a3efb4722a0b5b86ad2212.wav,12.068965517241379,14.459770114942529
conf is 0.9976705
avg above 1 is 0.9955006621100686
mixture_devtrain_gunshot_177_ccb69c71b8b7d64c00a4f25c13cbb620.wav,3.0344827586206895,4.137931034482759
conf is 0.9947876
avg above 1 is 0.9563693505141043
mixture_devtrain_gunshot_178_d1e825e6aadc8399119bf7a0c54ce289.wav,23.080459770114942,24.942528735632184
conf is 0.85956365
avg above 1 is 0.573220807010109
mixture_devtrain_gunshot_178_d1e825e6aadc8399119bf7a0c54ce289.wav,23.080459770114942,24.942528735632184
conf is 0.82161784
avg above 1 is 0.5190352489550908
mixture_devtrain_gunshot_178_d1e825e6aadc8399119bf7a0c54ce289.wav,23.080459770114942,24.942528735632184
conf is 0.9979785
avg above 1 is 0.8689188130554699
mixture_devtrain_gunshot_180_95fddacce890375d21b67d1510b9f40c.wav,

...

mixture_devtrain_gunshot_372_647349749e7d7bd8f0ca73f6be942365.wav,11.816091954022989,12.758620689655173
conf is 0.9457324
avg above 1 is 0.7685566862875765
mixture_devtrain_gunshot_372_647349749e7d7bd8f0ca73f6be942365.wav,11.816091954022989,12.758620689655173
conf is 0.8671736
avg above 1 is 0.31726441133854
mixture_devtrain_gunshot_372_647349749e7d7bd8f0ca73f6be942365.wav,11.816091954022989,12.758620689655173
conf is 0.8839006
avg above 1 is 0.8081843703985214
mixture_devtrain_gunshot_375_ce82b0156931b2a00f9f3d6a8e524f08.wav,1.5632183908045976,2.160919540229885
conf is 0.5754014
avg above 1 is 0.2631423305720091
mixture_devtrain_gunshot_375_ce82b0156931b2a00f9f3d6a8e524f08.wav,1.5632183908045976,2.160919540229885
conf is 0.98352134
avg above 1 is 0.8162303392006003
mixture_devtrain_gunshot_375_ce82b0156931b2a00f9f3d6a8e524f08.wav,17.862068965517242,18.4367816091954
conf is 0.87580085
avg above 1 is 0.3281113796290897
mixture_devtrain_gunshot_377_9748fe16123122c0735f07ed6abd4813.wav,19

mixture_devtrain_gunshot_425_7863f3fcb2521c88cc321ce4a3e6a50f.wav,11.471264367816092,12.068965517241379
conf is 0.99628323
avg above 1 is 0.8619725987315178
mixture_devtrain_gunshot_425_7863f3fcb2521c88cc321ce4a3e6a50f.wav,11.471264367816092,12.068965517241379
conf is 0.9827644
avg above 1 is 0.7274142239242792
mixture_devtrain_gunshot_425_7863f3fcb2521c88cc321ce4a3e6a50f.wav,11.471264367816092,12.068965517241379
conf is 0.993238
avg above 1 is 0.7166855022819205
mixture_devtrain_gunshot_425_7863f3fcb2521c88cc321ce4a3e6a50f.wav,11.471264367816092,12.068965517241379
conf is 0.95532537
avg above 1 is 0.8934674210507761
mixture_devtrain_gunshot_425_7863f3fcb2521c88cc321ce4a3e6a50f.wav,16.620689655172416,17.333333333333332
conf is 0.9948742
avg above 1 is 0.8322001984342933
mixture_devtrain_gunshot_425_7863f3fcb2521c88cc321ce4a3e6a50f.wav,16.620689655172416,17.333333333333332
conf is 0.70864743
avg above 1 is 0.7172224966102633
mixture_devtrain_gunshot_425_7863f3fcb2521c88cc321ce4a3e6a50f.

mixture_devtrain_gunshot_461_4b108128a595ec61682a78483670b158.wav,2.6206896551724137,3.0344827586206895
conf is 0.8696664
avg above 1 is 0.44480479889830876
mixture_devtrain_gunshot_462_93236187e17891695211e26fa9e622f3.wav,4.275862068965517,5.379310344827586
conf is 0.9766376
avg above 1 is 0.8124680735636503
mixture_devtrain_gunshot_462_93236187e17891695211e26fa9e622f3.wav,4.275862068965517,5.379310344827586
conf is 0.7305503
avg above 1 is 0.8251419961452484
mixture_devtrain_gunshot_464_7e88aaa1daec18180b4d5ef602009cc6.wav,8.781609195402298,10.64367816091954
conf is 0.9768857
avg above 1 is 0.38160117106004193
mixture_devtrain_gunshot_465_d14de480079da90ed9aa16aeb28cdbf2.wav,3.908045977011494,5.080459770114943
conf is 0.99859625
avg above 1 is 0.9741043215448206
mixture_devtrain_gunshot_466_791d156323ab34a3d4df6d50c6885d1d.wav,1.5632183908045976,2.9195402298850572
conf is 0.99682057
avg above 1 is 0.9795103557963869
mixture_devtrain_gunshot_467_ac8836bb8dae3fd470eceac2633eb6c0.wav,15

mixture_devtrain_gunshot_497_154bd24df948ce6aeee88b5db161f53e.wav,18.850574712643677,20.436781609195403
conf is 0.9989536
avg above 1 is 0.9838714569301928
mixture_devtrain_gunshot_497_154bd24df948ce6aeee88b5db161f53e.wav,18.850574712643677,20.436781609195403
conf is 0.96362233
avg above 1 is 0.3310559999663383


IndexError: list index out of range

In [29]:
import math

In [34]:
def check_event_truth(target_name,start_time):
    with open('event_list_evaltest_gunshot.csv', mode='r') as csv_file:
        reader = list(csv.reader(csv_file, delimiter='\n'))
        
        for row,i in zip(reader,range(500)):
            fields= str(row).split('\\t')
            if len(fields)>1:
                
                if fields[0].replace('[','').replace("'",'') == target_name and fields[3].replace(']','').replace("'",'')==event and abs(float(fields[1])-float(start_time))<=0.5:
                    return True
    
    return False

In [35]:
#gets a file name and return the number in it!
def get_number_from_filename(s):
    p = s.split('_')
    return p[3]

In [20]:
#creating file with probability feature values
event = 'gunshot'
test_path = '../Maskrcnn_audio/datasets/eval_500_gunshot_44100_128/'
with open('devtest_regions_melspec_44100_128_37_models_features_2cnn_2LSTM.csv','w') as w_file:
    w_writer = csv.writer(w_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    w_writer.writerow(['file_name', 'start', 'finish','min_val', 'max_val','average','average of probs>=0.9',
                       'average of probs>=0.8','average of probs>=0.7','average of probs>=0.5','average of probs>=0.1'
                       ,'average over all','True/False','Mask-rcnn confidence','prob>0.9+conf','0.7*prob9+0.3conf',
                       '0.3*prob9+0.7conf','0.7*prob1+0.3*conf',
                               '0.3*prob1+0.7*conf'])
    
    for element in l:
        parts = element.split(';')
        
        if(parts[3]==event):
            current_file_name = parts[0].replace('[','').replace("'",'')
            start_time = float(parts[1])
            finish_time = float(parts[2])
            
            
            test_frames = create_melspec_test(test_path + current_file_name +'.png', start_time, finish_time)
            
            
            test_frames = test_frames.reshape(1,128,200,1)
            print(test_frames.shape)
            
            
            predict_values = []
            predictions = my_model.predict(x=test_frames)
            
            
            print(parts[0]+ parts[1]+parts[2])
            
            predict_values = predictions.tolist()
            
            file_number = get_number_from_filename(current_file_name)
            min_val = str(min(predict_values[0]))
            max_val = str(max(predict_values[0]))
            avg_val = str(sum(predict_values[0])/12)
            sum_above_9 = 0
            num_of_probs_above_9 = 0
            sum_above_8 = 0
            num_of_probs_above_8 = 0
            sum_above_7 = 0
            num_of_probs_above_7 = 0
            sum_above_5 = 0
            num_of_probs_above_5 = 0
            sum_above_1 = 0
            num_of_probs_above_1 = 0
            sum_over_all = 0
            avg_over_all = 0
            
            for i in range(len(predict_values[0])):
                sum_over_all += predict_values[0][i]
                if predict_values[0][i]>= 0.9:
                    sum_above_9 += predict_values[0][i]
                    num_of_probs_above_9 += 1
                if predict_values[0][i]>= 0.8:
                    sum_above_8 += predict_values[0][i]
                    num_of_probs_above_8 += 1
                if predict_values[0][i]>= 0.7:
                    sum_above_7 += predict_values[0][i]
                    num_of_probs_above_7 += 1
                if predict_values[0][i]>= 0.5:
                    sum_above_5 += predict_values[0][i]
                    num_of_probs_above_5 += 1
                if predict_values[0][i]>= 0.1:
                    sum_above_1 += predict_values[0][i]
                    num_of_probs_above_1 += 1
                   
            if num_of_probs_above_9 > 0:
                avg_above_9 = float(sum_above_9 / num_of_probs_above_9)
            else:
                avg_above_9 = 0
            if num_of_probs_above_8 > 0:
                avg_above_8 = float(sum_above_8 / num_of_probs_above_8)
            else:
                avg_above_8 = 0
            if num_of_probs_above_7 > 0:
                avg_above_7 = float(sum_above_7 / num_of_probs_above_7)
            else:
                avg_above_7 = 0
            if num_of_probs_above_5 > 0:
                avg_above_5 = float(sum_above_5 / num_of_probs_above_5)
            else:
                avg_above_5 = 0
            if num_of_probs_above_1 > 0:
                avg_above_1 = float(sum_above_1 / num_of_probs_above_1)
            else:
                avg_above_1 = 0
            avg_over_all = float(sum_over_all/len(predict_values[0]))
            
            region_truth = check_event_truth(current_file_name,parts[1])
            w_writer.writerow([file_number,parts[1],parts[2] , min_val , max_val , avg_val,avg_above_9, 
                               avg_above_8, avg_above_7, avg_above_5,avg_above_1,avg_over_all,region_truth,parts[4],
                               avg_above_9+float(parts[4]),0.7*avg_above_9+(0.3*float(parts[4])),
                               0.3*avg_above_9+(0.7*float(parts[4])),0.7*avg_above_1+(0.3*float(parts[4])),
                               0.3*avg_above_1+(0.7*float(parts[4]))])
            
            
            
        else:
            pass
            
            
w_file.close()

IndexError: list index out of range